In [1]:
import numpy as np
from scipy import stats
import pandas as pd
import seaborn as sns 
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn import metrics

sns.set(font_scale=1.2)
sns.set_style('whitegrid')
#%matplotlib inline

In [2]:
model_testing = 'y'
data_checks = 'n' # y -> visualize data and correlation plots (it takes a while to execute the merged pairplot)
# preproc_log = 'n' # se lo accendo non impara più nulla, con lo Z invece funziona
remove_outliers = True

## TRAINING
nepochs = 800
# choosen_optimizer = tf.keras.optimizers.Adam(learning_rate=0.001),
choosen_optimizer = tf.keras.optimizers.RMSprop(learning_rate=0.001)

# we use "Sparse" because classes aren't represented with one hot encoding ie (0,1) e (1,0) but with integers 0 e 1
# choosen_loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
choosen_loss = 'sparse_categorical_crossentropy'
 
activation = 'sigmoid'


# hyperparameters: learning_rate, batch_size, 
# choice of the optimizer (algorithm)

# Features W -> e + nu
Run number of the event, Event number, pt eta phi are the electron's kinematics variables, Q is electron's charge, type is where they found the electron (in the barrel or the endcap), the del- features and sigmaEtaEta are the differences between the track variable and the one of the cluster, HoverE is the ratio of the electron's energy measured HCAL/ECAL, iso are parameters associated to electron detection, MET features refer to the kinematics of the neutrino

In [3]:
dfenu_raw = pd.read_csv('~/Documents/tesi/thesis_notebooks/input_datasets/Wenu.csv')
dfenu_raw['type'] = dfenu_raw['type'].map({'EB': 0, 'EE': 1})

In [4]:
if data_checks=='y':
    dfenu_raw.info()
    dfenu_raw.head()

In [5]:
# to uniform datasets
dfenu_raw['iso'] = dfenu_raw.apply(lambda x: x['isoTrack'] + x['isoEcal'] + x['isoHcal'],axis=1)
if data_checks=='y':
    dfenu_raw.head(3)

## Data visualization, W -> e + nu
We will introduce `dfenu_scaled` (numpy array) and `dfenu_array_to_dfenu` (pandas dataframe) for the visualization WITH OUTLIERS, while `dfenu_filtered` and `dfenu_scaled_filtered` are for the visualization WITHOUT OUTLIERS

In [6]:
if data_checks=='y':
    scaler = preprocessing.MinMaxScaler()
    dfenu_scaled = scaler.fit_transform(dfenu_raw)
    dfenu_array_to_dfenu = pd.DataFrame(dfenu_scaled, columns = dfenu_raw.columns)
    # aesthetics
    plt.figure(figsize=(20, 10)) 
    sns.set(font_scale=1.0)
    # boxplot
    sns.boxplot(data=dfenu_array_to_dfenu, orient="h", whis=1.5)

**Boxplot comment**: `pt` e `MET` are the only ones that present *outliers* between the features of interest.

**The coloured rectangle covers the range 25-75 quantile, while the whiskers reach to 1.5 times the distance between 25-75th percentile in both directions (starting from the median)**

In [7]:
if data_checks=='y':
    fig, ax = plt.subplots(figsize=(20,20))
    # sns.set(font_scale=1.0)
    sns.heatmap(dfenu_array_to_dfenu.corr() , annot= True, linewidths=2, ax=ax, cmap='mako')

## Removing outliers

### first from visualization…
introducing `cutoff_scal_pt_e` and `cutoff_scal_MET_e` using quantiles (for coherence with the visualization using quantiles through boxplot)

In [8]:
if data_checks == 'y':
    cutoff_scal_pt_e = dfenu_array_to_dfenu['pt'].quantile(0.95)
    cutoff_scal_MET_e = dfenu_array_to_dfenu['MET'].quantile(0.95)
    
    dfenu_filtered = dfenu_array_to_dfenu[(dfenu_array_to_dfenu['pt']<cutoff_scal_pt_e) & (dfenu_array_to_dfenu['MET']<cutoff_scal_MET_e)]
    print("Total number of data points:\t\t" + str(len(dfenu_array_to_dfenu)))
    # for troubleshooting
    # excluded_e = dfenu_array_to_dfenu[(dfenu_array_to_dfenu['pt']>cutoff_scal_pt_e) | (dfenu_array_to_dfenu['MET']>cutoff_scal_MET_e)]
    # print(len(excluded_e))
    print("Data points after filtering outliers:\t " + str(len(dfenu_filtered)))

In [9]:
if data_checks=='y':
    # boxplot again without the outliers
    dfenu_scaled_filtered = scaler.fit_transform(dfenu_filtered)
    dfenu_filtered = pd.DataFrame(dfenu_scaled_filtered, columns = dfenu_filtered.columns)
    # aesthetics
    plt.figure(figsize=(20, 10)) 
    sns.set(font_scale=1.0)
    # boxplot
    sns.boxplot(data=dfenu_filtered, orient="h")

**NEW boxplot comment**: pt still shows data points out of the whiskers but due to their number, the results of the training, and the sake of keeping as much data as possible they will be kept and considered throughout the analysis

### …then from the real dataframe

In [10]:
dfe = dfenu_raw[['pt','eta','phi','Q','iso','MET','phiMET']]

if remove_outliers:
    cutoff_pt_e = dfe['pt'].quantile(.95)
    cutoff_MET_e = dfe['MET'].quantile(.95)
    dfe = dfe[(dfe['pt']<cutoff_pt_e) & (dfe['MET']<cutoff_MET_e)]

# Adding 'class' because of dataset merging 
class_col = []
for i in range(len(dfe['pt'])):
    class_col.append('enu')
    
dfe.loc[:,'class'] = class_col
# len(dfe)

In [11]:
# check for higher order correlations (this will be repeated after merging the 2 different datasets)
if data_checks=='y':
    # Unnecessary features already discarded (pairplot on df_e, not df_enu)
    plt.figure(figsize=(20, 10))
    sns.set(font_scale=2.0)
    sns.pairplot(dfenu, diag_kind='kde') # stands for Kernel Density Estimation

**Pairplot comment**: the features don't exhibit evident patters, they all seem to carry independent information

# Features W -> mu + nu
**Feature comment**: `chisq` is divided by dof, the kinematics features come from fits on data so chisq abnormally high should just be left out of the training of the network, because even if they were decay events they exhibit a peculiar behaviour (like outliers) and the network is not designed to take care of such fine phenomena, `dxy` is the impact parameter, the `iso` parameter represents the threshold for the cluster to be identifyed as the trace/sign of an muon 

In [12]:
dfmunu_raw = pd.read_csv('~/Documents/tesi/thesis_notebooks/input_datasets/Wmunu.csv')

In [13]:
if data_checks=='y':
    dfmunu_raw.info()
    dfmunu_raw.sample(3)

Let's get rid of the events with an high chisq

In [14]:
chisq_th = 10 # threshold for acceptance of the data. It is a reduced chi squared!
not_trustworthy = dfmunu_raw[dfmunu_raw['chiSq'] > chisq_th]
len(not_trustworthy)

4802

In [15]:
dfmunu_raw.drop(not_trustworthy.index, inplace=True)
# print(len(df_munu[df_munu['chiSq']>100]))

## Data visualization, W -> mu + nu

We will introduce `dfmunu_scaled` (numpy array) and `dfmunu_array_to_dfenu` (pandas dataframe) for the visualization WITH OUTLIERS, while `dfmunu_filtered` and `dfmunu_scaled_filtered` are for the visualization WITHOUT OUTLIERS

In [16]:
if data_checks=='y':
    scaler = preprocessing.MinMaxScaler()
    dfmunu_scaled = scaler.fit_transform(dfmunu_raw)
    dfmunu_array_to_dfmunu = pd.DataFrame(dfmunu_scaled, columns = dfmunu_raw.columns)
    # aesthetics
    plt.figure(figsize=(20, 10)) 
    sns.set(font_scale=1.0)
    # boxplot
    sns.boxplot(data=dfmunu_array_to_dfmunu, orient="h")

**Boxplot comment**: `pt` and `MET` are the only relevant features that exhibit outliers

In [17]:
if data_checks=='y':
    fig, ax = plt.subplots(figsize=(20,20))
    sns.set(font_scale=1.0)
    sns.heatmap(dfmunu_array_to_dfmunu.corr() , annot= True, linewidths=3, ax=ax, cmap='mako')

**Heatmap comment**: no relevant (linear) correlations between the features manifest itself at this stage

## Removing outliers

### first from visualization…

In [18]:
if data_checks=='y':
    cutoff_scal_pt_mu = dfmunu_array_to_dfmunu['pt'].quantile(0.95)
    cutoff_scal_MET_mu = dfmunu_array_to_dfmunu['MET'].quantile(0.95)
    
    dfmunu_filtered = dfmunu_array_to_dfmunu[(dfmunu_array_to_dfmunu['pt']<cutoff_scal_pt_mu) & (dfmunu_array_to_dfmunu['MET']<cutoff_scal_MET_mu)]
    print("Total number of data points:\t\t" + str(len(dfmunu_array_to_dfmunu)))
    # for troubleshooting
    # excluded_mu = dfmunu_array_to_dfmunu[(dfmunu_array_to_dfmunu['pt']>cutoff_scal_pt_mu) | (dfmunu_array_to_dfmunu['MET']>cutoff_scal_MET_mu)]
    # print(len(excluded_mu))
    print("Data points after filtering outliers:\t" + str(len(dfmunu_filtered)))

In [19]:
if data_checks=='y':
    # boxplot again without outliers
    dfmunu_scaled_filtered = scaler.fit_transform(dfmunu_filtered)
    dfmunu_filtered = pd.DataFrame(dfmunu_scaled_filtered, columns = dfmunu_filtered.columns)
    # aesthetics
    plt.figure(figsize=(20, 10)) 
    sns.set(font_scale=1.0)
    # boxplot
    sns.boxplot(data=dfmunu_filtered, orient="h")

**NEW boxplot comment**: here `pt` still shows data out of the whiskers but they will not be considered outliers in the following analysis  

### … then from the real dataframe

In [20]:
# again in preparation for future merging
dfmu = dfmunu_raw[['pt','eta','phi','Q','iso','MET','phiMET']]

if remove_outliers:
    cutoff_pt_mu = dfmunu_raw['pt'].quantile(.95)
    cutoff_MET_mu = dfmunu_raw['MET'].quantile(.95)
    dfmu = dfmu[(dfmu['pt']<cutoff_pt_mu) & (dfmu['MET']<cutoff_MET_mu)]

class_col = []
for i in range(len(dfmu['pt'])):
    class_col.append('munu')
    
dfmu.loc[:,'class'] = class_col

In [21]:
if data_checks=='y':
    # Already discarded unnecessary features (pairplot on dfmu, not dfmunu_raw)
    plt.figure(figsize=(20, 10))
    sns.set(font_scale=2.0)
    sns.pairplot(dfmu, diag_kind='kde')

# Merging data

In [22]:
del dfenu_raw, dfmunu_raw # clean up

In [23]:
df = pd.concat([dfe,dfmu], ignore_index=True, sort=False)
df.sample(5)

,pt,eta,phi,Q,iso,MET,phiMET,class
135421,46.6129,-0.1539,-1.2488,1,18.3840,34.6064,-0.6721,munu
98322,25.4276,-0.6050,-1.9844,-1,2.6338,6.8626,2.8868,munu
162529,32.1379,-0.6763,1.5339,1,89.6808,13.3023,-2.1650,munu
48950,37.7950,0.4799,-1.1036,1,2.8121,23.9980,2.2496,enu
43240,59.8489,2.3550,1.2104,-1,13.2963,25.2738,-1.7519,enu


In [24]:
del dfe, dfmu # clean up

## Data checks

In [25]:
if data_checks=='y':
    # Per verificare che il dataset sia bilanciato
    sns.countplot(data=df,x='class')

In [26]:
if data_checks=='y':
    plt.figure(figsize=(20, 10))
    sns.set(font_scale=2.0)
    fig_pairplot = sns.pairplot(df, hue="class", palette="deep")
    fig_pairplot.savefig('pairplot_merged.png')

**Merged Pairplot**: questo ha un'ulteriore utilità pké nonostante io abbia verificato che nn ci sono dipendenze evidenti separatamente tra le features, potenzialmente potrebbero esserci modi semplici di classificare sla base di poche feature (magari lo scatter tra 2 feature separa nettamente le 2 classi e quindi il problema di classificazione è immediatamente risolto). Effettivamente nn sembrano esserci maniere evidenti di risolvere il problema di classificazione a mano

## Dataframes to *train&test* the model

In [27]:
# Drop unnecessary features
df.drop(['iso','Q'], inplace=True, axis=1)

# map class to int
df['class'] = df['class'].map({'enu': 0, 'munu': 1})

In [28]:
# split labels from data
X = df.drop(['class'], axis=1)
y = df['class']

del df

In [29]:
# train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=0) # this outputs pandas

In [30]:
X_train.head(5)

,pt,eta,phi,MET,phiMET
80623,64.0320,2.1130,2.6280,18.8360,-0.3797
25980,53.6347,1.2090,1.7389,18.7289,1.7811
88459,46.9111,2.2845,2.1415,15.4821,-0.0468
129340,40.0649,-1.4004,-3.1235,22.4505,-1.7030
171820,42.5995,-0.4345,-1.6350,34.9186,0.9710


# Feature engineering

**Standardizzazione** (0 mean and variance 1, but it's still a linear transformation, I'm not forcing the data to be normal!)

In [31]:
# Costruiamo il layer e informiamolo (ie ho solo 1 forma/STRUTTURA che adatto ai dati MA NN LO STO POPOLANDO!)
X_train_scaler = tf.keras.layers.Normalization(axis=-1) # axis=-1 gli dice come leggere il tensor ie in quale direz calcolare medie e varianze (qua banalmente se sle righe (axis=0) o sle colonne (axis=-1 o axis=1 fa li stès))
X_train_scaler.adapt(tf.convert_to_tensor(X_train)) # qua informiamo il layer sui dati, ie si calcola medie e varianze che userà poi

X_test_scaler = tf.keras.layers.Normalization(axis=-1)
X_test_scaler.adapt(tf.convert_to_tensor(X_test))

# Building and testing the model

In [32]:
def my_model():
   model = tf.keras.Sequential([
    X_train_scaler,
    tf.keras.layers.Dense(28, activation=activation),
    tf.keras.layers.Dense(12, activation=activation),
  # tf.keras.layers.Dense(6, activation=activation),
    tf.keras.layers.Dense(12, activation=activation),
    tf.keras.layers.Dense(28, activation=activation),
    tf.keras.layers.Dense(2, activation=tf.nn.softmax)       
   ])
   model.compile(
                 optimizer=choosen_optimizer,
       # usiamo "Sparse" perché le classes nn sono rappresentate con one hot encoding ie (0,1) e (1,0) ma con interi ie 0 e 1
                 loss = choosen_loss,
                 metrics=['accuracy'])
 
   return model

model = my_model()

In [ ]:
if model_testing=='y':
    history = model.fit(X_train, y_train, epochs=nepochs, validation_data=(X_test, y_test), batch_size=100)

Epoch 1/800
1413/1413 ━━━━━━━━━━━━━━━━━━━━ 1s 612us/step - accuracy: 0.5723 - loss: 0.6720 - val_accuracy: 0.6625 - val_loss: 0.5912
Epoch 2/800
1413/1413 ━━━━━━━━━━━━━━━━━━━━ 1s 542us/step - accuracy: 0.6666 - loss: 0.5899 - val_accuracy: 0.6658 - val_loss: 0.5883
Epoch 3/800
1413/1413 ━━━━━━━━━━━━━━━━━━━━ 1s 543us/step - accuracy: 0.6660 - loss: 0.5881 - val_accuracy: 0.6696 - val_loss: 0.5852
Epoch 4/800
1413/1413 ━━━━━━━━━━━━━━━━━━━━ 1s 543us/step - accuracy: 0.6717 - loss: 0.5829 - val_accuracy: 0.6814 - val_loss: 0.5745
Epoch 5/800
1413/1413 ━━━━━━━━━━━━━━━━━━━━ 1s 549us/step - accuracy: 0.6884 - loss: 0.5664 - val_accuracy: 0.7357 - val_loss: 0.5271
Epoch 6/800
1413/1413 ━━━━━━━━━━━━━━━━━━━━ 1s 543us/step - accuracy: 0.7468 - loss: 0.5111 - val_accuracy: 0.7565 - val_loss: 0.5027
Epoch 7/800
1413/1413 ━━━━━━━━━━━━━━━━━━━━ 1s 545us/step - accuracy: 0.7534 - loss: 0.4958 - val_accuracy: 0.7577 - val_loss: 0.4914
Epoch 8/800
1413/1413 ━━━━━━━━━━━━━━━━━━━━ 1s 547us/step - accuracy: 

In [ ]:
if model_testing=='y':
    model.summary()

## Performance check

In [ ]:
if model_testing=='y':
    # aesthetics
    font = {'family' : 'chalkboard',
            'weight' : 'bold',
            'size'   : 18}
    
    plt.rc('font', **font)
    
    fig = plt.figure(figsize=(25,10))
    ax1 = plt.subplot(121) # qsto sta per 1,2,1 ie nrows, ncol, index
    ax2 = plt.subplot(122)
    
    # what to plot on first figure (ax1)
    ax1.plot(history.history['accuracy'], label='Training accuracy') # qua specifico cosa plottare (le y, le x sono implicite in history)
    ax1.plot(history.history['val_accuracy'], label = 'Validation accuracy') 
    
    ax1.set_title("Training and validation accuracy")
    ax1.set(xlabel='epoch', ylabel='Accuracy')
    ax1.legend(loc='lower right')
    
    # what to plot on first figure (ax1)
    ax2.plot(history.history['loss'], label='Training loss')
    ax2.plot(history.history['val_loss'], label='Validation loss')
    
    ax2.set_title("Training and validation loss")
    ax2.set(xlabel='epoch', ylabel='loss')
    ax2.legend(loc='upper right')
    
    #To check the network accuracy on test data
    test_loss, test_acc = model.evaluate(X_test,  y_test, verbose=2)

In [ ]:
#fig.savefig('28_12_6_12_28_Adam.png')